In [1]:
import pandas as pd ##Data handeler
import numpy as np ##Maths functions
import seaborn as sns ##Graphing
import matplotlib ##Graphing
import matplotlib.pyplot as plt


from scipy.stats import skew
from scipy.stats.stats import pearsonr

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, LassoCV, LassoLarsCV, ElasticNet

from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

# Import Data Set

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Select Columns (remove sale price)

In [3]:
data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'], test.loc[:,'MSSubClass':'SaleCondition']))

# Take Log of Sale Price

In [4]:
train["SalePrice"] = np.log1p(train["SalePrice"])

# Create Variable Containing Columns

In [5]:
columns = data.dtypes[data.dtypes != "object"].index

# Removes feilds with no data

In [6]:
skewed = train[columns].apply(lambda y: skew(y.dropna()))
data = (pd.get_dummies(data)).fillna(data.mean())

In [7]:
skewed = skewed.index
data[skewed] = np.log1p(data[skewed])

# Put Sampled data into appropriate varaibles

In [8]:
X_train = data[:train.shape[0]]
X_test = data[train.shape[0]:]
y = train["SalePrice"]


# Root mean square function


In [9]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)
##Function calculates deviation from regression

# Lasso model

In [10]:
ml = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]) #tests which method produces the least mean square error
ml.fit(X_train, y) #produces regression line
predict_ml = ml.predict(X_test) #predicts house price
ml_pred = np.expm1(predict_ml) #unlogs all house prices

# Linear Regression Algorithm

In [11]:
linearRegression = LinearRegression()
linearRegression.fit(X_train, y)
linearRegression.score(X_train, y)

0.9464184973842314

In [12]:
predict_LR = linearRegression.predict(X_test)
li_pred = np.expm1(predict_LR)

# Create Submission File 

In [13]:
Id = []
for i in range(1461, 2920, 1) : Id.append(i)
submission = pd.DataFrame({'ID': Id, 'SalePrice': ml_pred})
submission.to_csv('submission.csv', index=False)